In [175]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [176]:
import os
os.chdir('/content/gdrive/MyDrive/Project')

In [177]:
!pwd

/content/gdrive/MyDrive/Project


In [178]:
# Common imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re

%matplotlib inline

# Import Scikit-learn helper functions
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Import Scikit-learn models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# Import Scikit-learn metric functions
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.metrics import accuracy_score

In [179]:
# Import Scikit-learn models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# Import Scikit-learn metric functions
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.metrics import accuracy_score

In [180]:
url =  pd.read_csv('urldata.csv')

In [181]:
url.head()

,Unnamed: 0,url,label,result
0,0,https://www.google.com,benign,0
1,1,https://www.youtube.com,benign,0
2,2,https://www.facebook.com,benign,0
3,3,https://www.baidu.com,benign,0
4,4,https://www.wikipedia.org,benign,0


In [182]:
def makeTokens(f):
    tkns_BySlash = str(f.encode('utf-8')).split('/')	# make tokens after splitting by slash
    total_Tokens = []
    for i in tkns_BySlash:
        tokens = str(i).split('-')	# make tokens after splitting by dash
        tkns_ByDot = []
        for j in range(0,len(tokens)):
            temp_Tokens = str(tokens[j]).split('.')	# make tokens after splitting by dot
            tkns_ByDot = tkns_ByDot + temp_Tokens
        total_Tokens = total_Tokens + tokens + tkns_ByDot
    total_Tokens = list(set(total_Tokens))	#remove redundant tokens
    if 'com' in total_Tokens:
        total_Tokens.remove('com')	#removing .com since it occurs a lot of times and it should not be included in our features
    return total_Tokens

In [183]:
x = url['url']
y = url['label']

In [184]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

In [185]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.3,shuffle=True,random_state=5)

In [186]:
logit = LogisticRegression()
logit.fit(x_train, y_train)

LogisticRegression()

In [187]:
logit.fit(x_train, y_train)

LogisticRegression()

In [188]:
x_train_prediction = logit.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [189]:
print('The Accuracy score of the training data is: ', training_data_accuracy*100)

The Accuracy score of the training data is:  99.62078299584607


In [190]:
# accuracy score on the test data
x_test_prediction = logit.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [191]:
print('The Accuracy score of the test data is : ', test_data_accuracy*100)

The Accuracy score of the test data is :  99.49945576921652


In [192]:
y_pred = logit.predict(x_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 99.5%


In [193]:
begnign=0
Malicious =1

In [194]:
cf = confusion_matrix(y_test,y_pred, labels=['malicious','benign'])
print(cf)

[[ 30661    670]
 [     6 103716]]


In [195]:
def url_det(url):
    input_data = [url]
    vectorized_input_data = vectorizer.transform(input_data)
    prediction = logit.predict(vectorized_input_data)
    print('This url appears to be', prediction)

In [196]:
url_det('https://www.wikipedia.org')

This url appears to be ['benign']


In [203]:
import requests
from bs4 import BeautifulSoup
from difflib import SequenceMatcher
import openpyxl


In [198]:
#ask user if they will like to get a list of similar urls

In [199]:
similar_url_option= input("would you like to get a list of similar urls? Enter 'yes' or 'no': ")

would you like to get a list of similar urls? Enter 'yes' or 'no': yes


In [200]:
user_wants_url = False
if similar_url_option == 'no':
  print("Thank you for using Dami's url predictor, Have a great day !")

else:
   sitemap_url = input("Enter sitemap URL: ")



Enter sitemap URL: https://www.kaggle.com/


In [201]:
try:
    # Send a GET request to the sitemap URL and retrieve the response
    response = requests.get(sitemap_url)
    response.raise_for_status()

    # Parse the XML content of the sitemap using BeautifulSoup
    soup = BeautifulSoup(response.content, "xml")

    # Take user input for base URL
    #base_url = input("Enter base URL: ")

    # Find all the URL elements in the sitemap
    url_elements = soup.find_all("url")

    # Define a function to compute the similarity rate between two URLs
    def compute_similarity(url1, url2):
        # Extract the paths of the URLs after the base URL
        path1 = url1[len(sitemap_url):]
        path2 = url2[len(sitemap_url):]

        # Compute the similarity rate between the two paths
        return SequenceMatcher(None, path1, path2).ratio()

    # Take user input for similarity threshold
    similarity_threshold = float(input("Please enter a similarity threshold between values of 0.1 and 1: "))

    # Create a new Excel workbook and add a worksheet for the results
    workbook = openpyxl.Workbook()
    worksheet = workbook.active
    worksheet.title = "Similar URLs"

    # Add headers to the worksheet
    worksheet["A1"] = "URL 1"
    worksheet["B1"] = "URL 2"
    worksheet["C1"] = "Similarity rate"

    # Loop through each pair of URLs and compute the similarity rate
    row_index = 2
    found_similar_urls = False
    for i in range(len(url_elements)):
        url1 = url_elements[i].find("loc").text
        for j in range(i+1, len(url_elements)):
            url2 = url_elements[j].find("loc").text
            similarity_rate = compute_similarity(url1, url2)
            if similarity_rate >= similarity_threshold:
                worksheet.cell(row=row_index, column=1, value=url1)
                worksheet.cell(row=row_index, column=2, value=url2)
                worksheet.cell(row=row_index, column=3, value=similarity_rate)
                row_index += 1
                found_similar_urls = True

    if not found_similar_urls:
        print("No pairs of URLs found with similarity rate above the threshold.")
    else:
        # Ask the user whether they want to print the results or export to Excel
        export_option = input("Do you want to print the results or export them to an Excel file? Enter 'print' or 'excel': ")
        if export_option == "print":
            # Print the results directly
            for row in worksheet.iter_rows(values_only=True):
                print("\t".join(str(cell) for cell in row))
        elif export_option == "excel":
            # Save the results to an Excel file
            excel_filename = "similar_urls.xlsx"
            workbook.save(excel_filename)
            print(f"Results saved to {excel_filename}.")
        else:
            print("Invalid option. Please enter 'print' or 'excel'.")

except Exception as e:
    print(f"Error retrieving sitemap: {e}")

Please enter a similarity threshold between values of 0.1 and 1: 0.3
No pairs of URLs found with similarity rate above the threshold.
